## Modeling

---

EXPLICAR POR QUE ESTOS MODELOS Y COMO LO HACE CADA UNO Y COMO SE VALIDA Y LA PARTE DE POR QUE LA METRICA ES F1

# 2) 📦 Datos y Data Readiness

**Versionado y tamaños (train/test)**  
- Rutas: `../data/processed/train_posts_clean.csv` y `../data/processed/test_posts_clean.csv`.  
- Tamaños usados en entrenamiento/final: **Train (685, 11)** | **Test (172, 11)**.  
- Integridad: en cada run se loggea `md5` y `shape` de ambos datasets como artifacts (`*_train_meta.json`, `*_test_meta.json`).

**Target & métricas**  
- **Target:** `score_clipped` (recorta outliers vía IQR para estabilizar la varianza).  
- **Métrica principal:** **RMSE** (menor es mejor).  
- Métrica reportada en **val** durante HPO y en **test** para el modelo final.

**Selección de variables (features)**  
Se parte de un set curado por EDA y de ingeniería ligera:  
- Numéricas: `num_comments_capped`, `recency_days`, `dayofweek`, `title_len`, `selftext_len`, `is_self`, `month`  
- Categóricas: `link_flair_text`, `subreddit`, `author`  
*(el código filtra automáticamente por columnas presentes en el dataset).*

**Prevención de data leakage**  
- Se **excluyen** columnas con fuga: `post_id`, `url`, `permalink`, `thumbnail`.  
- El target usa `score_clipped` (no derivado de variables futuras).  
- La evaluación final se hace en **test** separado.

**Tabla breve de features (tipo y razón)**

| Feature              | Tipo      | Razón/hipótesis de valor |
|---|---|---|
| `num_comments_capped` | Numérica  | Señal de interacción temprana; cap al p99 reduce var extrema. |
| `recency_days`       | Numérica  | Efecto de frescura del post. |
| `dayofweek`          | Numérica (entero) | Patrones de consumo por día. |
| `month`              | Numérica (entero) | Estacionalidad y tendencias. |
| `title_len`          | Numérica  | Longitud del título correlaciona con CTR/engagement. |
| `selftext_len`       | Numérica  | Carga informativa del contenido. |
| `is_self`            | Binaria   | Diferencia entre link-post y text-post. |
| `link_flair_text`    | Categórica| Tema/contexto del post (moderación/comunidad). |
| `subreddit`          | Categórica| Efecto de comunidad. |
| `author`             | Categórica| Efecto autor (historial/credibilidad). |

---

# 3) 🧱 Preprocesamiento (ColumnTransformer/Pipeline)

**Definición**  
- `ColumnTransformer` con:  
  - **Numéricas:** `StandardScaler(with_mean=True, with_std=True)` sobre columnas numéricas.  
  - **Categóricas:** `OneHotEncoder(handle_unknown="ignore", min_frequency=5)` sobre `object`.

**Integración y versionado**  
- El **preprocessor está integrado** en cada `Pipeline` (`("prep", preprocessor) → ("model", estimador)`), por lo que **viaja junto al modelo**.  
- Además, se **versiona por separado**:  
  - Se loggea como **artifact MLflow** en `preprocessor/`.  
  - Se guarda **local** en `../preprocesador/<modelo>_preprocessor_<timestamp>/` (directorio con `MLmodel` + pickles).

**Reproducibilidad**  
- Semilla global `SEED=42` para split y modelos; `Hyperopt` con `default_rng(SEED)`.  
- Tracking determinístico: se registran rutas, shapes, `md5`, lista de `FEATURES` y `TARGET` en `run_context.txt`.

**Evidencia mínima**  
- En los runs `FINAL_*` aparecen artifacts `model/` (pipeline completo) y `preprocessor/` (solo transformador).  
- Mensajes de consola confirman los guardados locales en `../preprocesador/...`.

---

# 4) 🧪 Experimentos de Modelado (≥3 modelos)

**Familias y HPO**  
- Modelos: **ElasticNet**, **RandomForestRegressor**, **XGBRegressor**.  
- Para cada familia:  
  - `HPO_<modelo>` con **runs anidados** (`nested=True`).  
  - Se loggean **parámetros**, **RMSE (val)**, **snapshots** (`<modelo>_best_snapshot`) y **metadatos de datasets**.  

**Evidencia (capturas MLflow)**  
- Se observan los runs `HPO_elasticnet`, `HPO_random_forest`, `HPO_xgboost` y sus `*_trial`.  
- En la tabla de experimentos se comparan métricas y parámetros por run.

---

# 5) 🎯 Tuning con Hyperopt + MLflow (Databricks)

- **Espacios de búsqueda** definidos por familia (p. ej., `n_estimators`, `max_depth`, `l1_ratio`, etc.).  
- **Objetivo:** minimizar **RMSE** en `val`.  
- **Estructura de runs:**  
  - `IMEDIA_EMI_AllModels` (parent) → `HPO_<modelo>` (parent por familia) → `<modelo>_trial` (runs hijos).  
- **Artifacts clave:** `run_context.txt`, snapshots de los mejores pipelines por familia y metadatos de datasets.

**Evidencia mínima**  
- Pantalla de MLflow muestra **runs anidados** y **parámetros**; se ve la **comparativa** de RMSE por familia.

---

# 6) 🏆 Selección del Mejor Modelo

**Criterio**  
- Orden explícito por **RMSE (ascendente)** sobre la validación: `results_df.sort_values("val_rmse")`.

**Resultado**  
- Ranking en `val`: **Random Forest (210.75)** < ElasticNet (216.74) < XGBoost (220.67).  
- Se reentrena el **mejor** con todo el train y se evalúa en **test**.

**Evidencia de “mejor en contexto”**  
- En **test**, **Random Forest** consigue **RMSE ≈ 171.82**, muy por debajo del segundo (**ElasticNet ≈ 248.97**).  
- Justificación breve: RF captura no linealidades e interacciones típicas entre señales de interacción (`num_comments_capped`) y contexto (`subreddit`, `flair`), mejor que modelos lineales; XGB quedó atrás con este tamaño y sparsidad.

**Identificador de best run**  
- Run final: `FINAL_random_forest_on_full_train` (véase MLflow UI en el experimento).  
- Tabla comparativa exportada: `model_metric_comparison.csv`.

---

# 7) 🗂️ Registro en el Model Registry

**Nomenclatura**  
- Modelo en UC: **`workspace.default/EMI_imedia_model`**.

**Qué se registra**  
- **Pipeline completo** (incluye `preprocessor` dentro del `Pipeline`).  
- **Alias:**  
  - `@champion` → **Version 3** (Random Forest, **test_rmse ≈ 171.82**).  
  - `@challenger` → **Version 4** (ElasticNet, **test_rmse ≈ 248.97**).

**Documentación del modelo**  
- En `model_description.json` se incluyen: datos, métrica, fecha, features, seeds, y changelog.  
- **Preprocessor** adicionalmente guardado como artifact `preprocessor/` y **local** en `../preprocesador/…`.

**Evidencia mínima**  
- Capturas de **Catalog Explorer** muestran `emi_imedia_model` con **Version 3 (@champion)** y **Version 4 (@challenger)**.  
- En el listado del experimento se visualizan `FINAL_*` y la **métrica de test** de cada final.

---

## 📊 Tabla comparativa (RMSE ↓)

| model_key     | split | RMSE |
|---|---:|---:|
| random_forest | test | **171.82** |
| elasticnet    | test | 248.97 |
| random_forest | val  | 210.75 |
| elasticnet    | val  | 216.74 |
| xgboost       | val  | 220.67 |

> **Conclusión:** Se promueve **`workspace.default/EMI_imedia_model@champion (v3)`**. El **preprocessor** queda **dentro** del pipeline registrado y **aparte** como artifact + copia **local** en `../preprocesador/`.


In [26]:
# 1. Setup general, carga de datos, MLflow y helpers de artefactos (NTBK)

import os, json, warnings
warnings.filterwarnings("ignore")

from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd

import mlflow
from mlflow import MlflowClient
from mlflow.models.signature import infer_signature

from dotenv import load_dotenv

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer

# ----------------------
# Configuración base
# ----------------------
SEED = 42
np.random.seed(SEED)

EXPERIMENT_NAME = "/Users/marianasgg19@gmail.com/EMI/imedia/Sentiment_BERT_MLP"
DEEP_MODEL_NAME = "workspace.default.EMI_imedia_sentiment_deep_model_NTBK"

DATA_DIR = Path("../data/processed")
TRAIN_PATH = DATA_DIR / "sentiment_train.parquet"
VAL_PATH   = DATA_DIR / "sentiment_val.parquet"
TEST_PATH  = DATA_DIR / "sentiment_test.parquet"

assert TRAIN_PATH.exists() and VAL_PATH.exists() and TEST_PATH.exists(), "Faltan splits en data/processed"

train_df = pd.read_parquet(TRAIN_PATH)
val_df   = pd.read_parquet(VAL_PATH)
test_df  = pd.read_parquet(TEST_PATH)

X_train_text = train_df["clean_text"].astype(str).tolist()
X_val_text   = val_df["clean_text"].astype(str).tolist()
X_test_text  = test_df["clean_text"].astype(str).tolist()

y_train = train_df["sentiment"].astype(int).values
y_val   = val_df["sentiment"].astype(int).values
y_test  = test_df["sentiment"].astype(int).values

# ----------------------
# Directorios de artefactos (raíz)
# ----------------------
EMBEDDINGS_DIR = Path("../embeddings")
EMBEDDINGS_DIR.mkdir(parents=True, exist_ok=True)

PREPROC_LOCAL_DIR = Path("../preprocesador")
PREPROC_LOCAL_DIR.mkdir(parents=True, exist_ok=True)

# ----------------------
# Lista global de resultados (para comparar modelos)
# ----------------------
RESULTS = []   # cada entrada: dict con model_key, run_id, val_f1, test_f1, etc.

# ----------------------
# Tracking URI resiliente
# ----------------------
def _resolve_tracking_uri() -> str:
    load_dotenv(override=True)
    env_uri = os.getenv("MLFLOW_TRACKING_URI")
    profile = os.getenv("DATABRICKS_CONFIG_PROFILE")
    host = os.getenv("DATABRICKS_HOST")
    token = os.getenv("DATABRICKS_TOKEN")

    if env_uri:
        return env_uri
    if profile:
        return f"databricks://{profile}"
    if host and token:
        return "databricks"

    local_store = Path.cwd() / "mlruns"
    local_store.mkdir(parents=True, exist_ok=True)
    return f"file://{local_store}"

def set_mlflow():
    tracking_uri = _resolve_tracking_uri()
    mlflow.set_tracking_uri(tracking_uri)
    client = MlflowClient()
    exp = client.get_experiment_by_name(EXPERIMENT_NAME)
    if exp is None:
        client.create_experiment(EXPERIMENT_NAME)
    mlflow.set_experiment(EXPERIMENT_NAME)

set_mlflow()

# ----------------------
# Helper: guardar embeddings como artefacto reproducible (sufijo NTBK)
# ----------------------
import datetime as dt

def save_embeddings_artifact(model_name: str,
                             embeddings_matrix: np.ndarray,
                             split: str,
                             extra_metadata: dict | None = None) -> dict:
    """
    Guarda embeddings + metadata en ../embeddings con sufijo NTBK.
    """
    safe_model_name = model_name.replace("/", "_")
    timestamp = dt.datetime.utcnow().strftime("%Y%m%dT%H%M%S")

    base_name = f"{safe_model_name}_{split}_{timestamp}_NTBK"

    emb_path = EMBEDDINGS_DIR / f"{base_name}_embeddings.npy"
    meta_path = EMBEDDINGS_DIR / f"{base_name}_metadata.json"

    np.save(emb_path, embeddings_matrix)

    metadata = {
        "model_name": model_name,
        "split": split,
        "embedding_shape": embeddings_matrix.shape,
        "created_utc": dt.datetime.utcnow().isoformat() + "Z",
    }
    if extra_metadata:
        metadata.update(extra_metadata)

    pd.Series(metadata).to_json(meta_path)

    return {
        "embeddings_path": str(emb_path),
        "metadata_path": str(meta_path),
    }


In [27]:
# 2. Dos modelos BERT para clasificación directa de sentimiento + MLflow + guardado de tokenizers (preprocesador/)

from scipy.special import softmax
import torch

BERT_MODELS = [
    "nlptown/bert-base-multilingual-uncased-sentiment",      # 5 clases (1-5 stars)
    "distilbert-base-uncased-finetuned-sst-2-english",       # 2 clases (neg/pos)
]

def predict_binary_from_logits(logits: np.ndarray, model_name: str) -> int:
    """
    Convierte logits de cada modelo a etiqueta binaria 0/1.
    Implementación simple basada en convenciones típicas de estos checkpoints.
    """
    probs = softmax(logits)[0]
    if "nlptown" in model_name:
        # 5 clases -> 1,2 = negativo; 3 = neutro; 4,5 = positivo
        idx = np.argmax(probs)
        return 1 if idx >= 3 else 0
    else:
        # Modelos binarios: índice 1 = positivo
        idx = np.argmax(probs)
        return 1 if idx == 1 else 0

def run_bert_model(model_name: str):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    def predict_batch(texts):
        preds = []
        for t in texts:
            inputs = tokenizer(
                t,
                return_tensors="pt",
                truncation=True,
                padding=True,
                max_length=256,
            )
            with torch.no_grad():
                logits = model(**inputs).logits.numpy()
            preds.append(predict_binary_from_logits(logits, model_name))
        return np.array(preds, dtype=int)

    with mlflow.start_run(run_name=f"BERT_{model_name.split('/')[-1]}") as run:
        y_val_pred = predict_batch(X_val_text)
        y_test_pred = predict_batch(X_test_text)

        val_acc = accuracy_score(y_val, y_val_pred)
        val_f1  = f1_score(y_val, y_val_pred, average="binary")
        test_acc = accuracy_score(y_test, y_test_pred)
        test_f1  = f1_score(y_test, y_test_pred, average="binary")

        mlflow.log_param("model_family", "bert_sequence_classifier")
        mlflow.log_param("hf_model_name", model_name)
        mlflow.log_metric("val_accuracy", val_acc)
        mlflow.log_metric("val_f1",       val_f1)
        mlflow.log_metric("test_accuracy", test_acc)
        mlflow.log_metric("test_f1",       test_f1)

        print(f"\n=== {model_name} ===")
        print("Validation accuracy:", val_acc)
        print("Validation F1      :", val_f1)
        print("Test accuracy      :", test_acc)
        print("Test F1            :", test_f1)

        # Guardar tokenizer como "preprocesador" en raíz/preprocesador con sufijo NTBK
        safe_name = model_name.replace("/", "_")
        ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
        tok_dir = PREPROC_LOCAL_DIR / f"{safe_name}_tokenizer_NTBK_{ts}"
        tok_dir.mkdir(parents=True, exist_ok=True)
        tokenizer.save_pretrained(tok_dir)

        # Loggear tokenizer como artifact en MLflow
        mlflow.log_artifacts(str(tok_dir), artifact_path="preprocessor_tokenizer")

        # ---> Guardar resultados en la tabla global RESULTS
        RESULTS.append({
            "model_key": f"bert_{model_name.split('/')[-1]}",
            "run_id": run.info.run_id,
            "val_accuracy": float(val_acc),
            "val_f1": float(val_f1),
            "test_accuracy": float(test_acc),
            "test_f1": float(test_f1),
        })

for m_name in BERT_MODELS:
    run_bert_model(m_name)



=== nlptown/bert-base-multilingual-uncased-sentiment ===
Validation accuracy: 0.46120689655172414
Validation F1      : 0.3902439024390244
Test accuracy      : 0.4454022988505747
Test F1            : 0.3713355048859935
🏃 View run BERT_bert-base-multilingual-uncased-sentiment at: https://dbc-5922e233-b716.cloud.databricks.com/ml/experiments/1690980704956355/runs/58dda8810b2b4149969404d9a8d8ce1e
🧪 View experiment at: https://dbc-5922e233-b716.cloud.databricks.com/ml/experiments/1690980704956355

=== distilbert-base-uncased-finetuned-sst-2-english ===
Validation accuracy: 0.4367816091954023
Validation F1      : 0.35737704918032787
Test accuracy      : 0.4238505747126437
Test F1            : 0.3500810372771475
🏃 View run BERT_distilbert-base-uncased-finetuned-sst-2-english at: https://dbc-5922e233-b716.cloud.databricks.com/ml/experiments/1690980704956355/runs/0b5158f11719429fa2796043612980b1
🧪 View experiment at: https://dbc-5922e233-b716.cloud.databricks.com/ml/experiments/169098070495635

In [28]:
# 3. SentenceTransformer embeddings + MLPClassifier + artefactos NTBK + registro en Model Registry
#    + comparación entre los 3 modelos y asignación de alias 'champion' si el mejor es el MLP

st_model_name = "sentence-transformers/all-MiniLM-L6-v2"
st_model = SentenceTransformer(st_model_name)

def embed_texts(model, texts, batch_size=32):
    texts = list(texts)
    emb = model.encode(
        texts,
        batch_size=batch_size,
        show_progress_bar=True,
        convert_to_numpy=True,
    )
    return emb

# ---- Embeddings para cada split ----
X_train_emb = embed_texts(st_model, X_train_text)
X_val_emb   = embed_texts(st_model, X_val_text)
X_test_emb  = embed_texts(st_model, X_test_text)

print("Emb shapes:", X_train_emb.shape, X_val_emb.shape, X_test_emb.shape)

# Guardar embeddings como artefactos NTBK en ../embeddings
emb_info_train = save_embeddings_artifact(st_model_name, X_train_emb, split="train")
emb_info_val   = save_embeddings_artifact(st_model_name, X_val_emb,   split="val")
emb_info_test  = save_embeddings_artifact(st_model_name, X_test_emb,  split="test")

# ---- Pipeline: scaler + MLPClassifier ----
mlp_clf = MLPClassifier(
    hidden_layer_sizes=(256,),
    activation="relu",
    solver="adam",
    batch_size=256,
    learning_rate_init=1e-3,
    max_iter=10,
    random_state=SEED,
    verbose=True,
)

pipe = Pipeline(steps=[
    ("scaler", StandardScaler(with_mean=True, with_std=True)),
    ("mlp", mlp_clf),
])

from mlflow import sklearn as mlflow_sklearn

with mlflow.start_run(run_name="mlp_transformer_embeddings_NTBK") as run:
    pipe.fit(X_train_emb, y_train)

    y_val_pred  = pipe.predict(X_val_emb)
    y_test_pred = pipe.predict(X_test_emb)

    val_acc = accuracy_score(y_val, y_val_pred)
    val_f1  = f1_score(y_val, y_val_pred, average="binary")
    test_acc = accuracy_score(y_test, y_test_pred)
    test_f1  = f1_score(y_test, y_test_pred, average="binary")

    print("\nMLP + Transformer - Validation accuracy:", val_acc)
    print("MLP + Transformer - Validation F1      :", val_f1)
    print("MLP + Transformer - Test accuracy      :", test_acc)
    print("MLP + Transformer - Test F1            :", test_f1)

    mlflow.log_param("model_family", "mlp_classifier")
    mlflow.log_param("features", "sentence_transformers_embeddings")
    mlflow.log_param("sentence_transformer_model", st_model_name)
    mlflow.log_param("hidden_layer_sizes", "(256,)")
    mlflow.log_param("activation", "relu")
    mlflow.log_param("solver", "adam")
    mlflow.log_param("batch_size", 256)
    mlflow.log_param("max_iter", 10)

    mlflow.log_param("train_emb_size", len(y_train))
    mlflow.log_param("val_emb_size",   len(y_val))
    mlflow.log_param("test_emb_size",  len(y_test))

    mlflow.log_metric("val_accuracy",  val_acc)
    mlflow.log_metric("val_f1",        val_f1)
    mlflow.log_metric("test_accuracy", test_acc)
    mlflow.log_metric("test_f1",       test_f1)

    # Log de embeddings como artifacts de MLflow (rutas creadas antes)
    mlflow.log_artifact(emb_info_train["embeddings_path"], artifact_path="embeddings/train")
    mlflow.log_artifact(emb_info_val["embeddings_path"],   artifact_path="embeddings/val")
    mlflow.log_artifact(emb_info_test["embeddings_path"],  artifact_path="embeddings/test")

    # Firma del modelo
    example_input = X_train_emb[:100]
    example_output = pipe.predict(example_input)
    signature = infer_signature(example_input, example_output)

    # --- Registry: Unity Catalog (nombre DEEP_MODEL_NAME ya es 3-niveles) ---
    mlflow.set_registry_uri("databricks-uc")

    mlflow.sklearn.log_model(
        sk_model=pipe,
        artifact_path="model",
        registered_model_name=DEEP_MODEL_NAME,
        signature=signature,
        input_example=example_input[:5],
    )

    # Guardar PREPROCESADOR (scaler) localmente con sufijo NTBK
    fitted_scaler = pipe.named_steps["scaler"]
    ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    local_preproc_dir = PREPROC_LOCAL_DIR / f"mlp_scaler_NTBK_{ts}"
    mlflow_sklearn.save_model(sk_model=fitted_scaler, path=str(local_preproc_dir))

    # Loggear también el preprocesador como artifact
    mlflow_sklearn.log_model(
        sk_model=fitted_scaler,
        artifact_path="preprocessor_scaler_NTBK",
    )

    # ---> Guardar resultados del MLP en la tabla global RESULTS
    RESULTS.append({
        "model_key": "mlp_transformer",
        "run_id": run.info.run_id,
        "val_accuracy": float(val_acc),
        "val_f1": float(val_f1),
        "test_accuracy": float(test_acc),
        "test_f1": float(test_f1),
    })

# ==========================
# Comparativa entre modelos
# ==========================
results_df = pd.DataFrame(RESULTS).sort_values("val_f1", ascending=False).reset_index(drop=True)
print("\n=== Comparación de modelos por val_f1 (desc) ===")
print(results_df)

best_row = results_df.iloc[0]
best_model_key = best_row["model_key"]
print(f"\n🏆 Mejor modelo según val_f1: {best_model_key} (val_f1={best_row['val_f1']:.4f})")

# ==========================
# Alias 'champion' en UC SOLO si el mejor es el MLP
# ==========================
from mlflow import MlflowClient

if best_model_key == "mlp_transformer":
    mlflow.set_registry_uri("databricks-uc")
    client = MlflowClient()

    # UC no soporta get_latest_versions -> usamos search_model_versions
    versions = client.search_model_versions(f"name = '{DEEP_MODEL_NAME}'")

    if versions:
        champ_version = max(int(v.version) for v in versions)
        client.set_registered_model_alias(
            name=DEEP_MODEL_NAME,
            alias="champion",
            version=champ_version,
        )
        print(f"\n✅ Alias 'champion' asignado a {DEEP_MODEL_NAME} (v{champ_version})")
    else:
        print(f"\n⚠ No se encontraron versiones para el modelo {DEEP_MODEL_NAME}")
else:
    print("\n⚠ El mejor modelo es un BERT (no registrado en UC en este notebook).")
    print("   El MLP se registró en el Model Registry, pero SIN alias 'champion'.")


Batches:   0%|          | 0/102 [00:00<?, ?it/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Emb shapes: (3245, 384) (696, 384) (696, 384)
Iteration 1, loss = 0.66617470
Iteration 2, loss = 0.51802273
Iteration 3, loss = 0.45699580
Iteration 4, loss = 0.41061293
Iteration 5, loss = 0.36952780
Iteration 6, loss = 0.33279928
Iteration 7, loss = 0.29810586
Iteration 8, loss = 0.26197197
Iteration 9, loss = 0.23120460
Iteration 10, loss = 0.20227363

MLP + Transformer - Validation accuracy: 0.6594827586206896
MLP + Transformer - Validation F1      : 0.6926070038910506
MLP + Transformer - Test accuracy      : 0.6910919540229885
MLP + Transformer - Test F1            : 0.7144754316069057


2025/12/02 03:50:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025/12/02 03:50:02 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'workspace.default.EMI_imedia_sentiment_deep_model_NTBK' already exists. Creating a new version of this model...


Uploading artifacts:   0%|          | 0/7 [00:00<?, ?it/s]

Created version '3' of model 'workspace.default.emi_imedia_sentiment_deep_model_ntbk'.
2025/12/02 03:50:13 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025/12/02 03:50:14 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/12/02 03:50:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/02 03:50:15 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
huggingface/tokenizers: The current process just got forked, after para

🏃 View run mlp_transformer_embeddings_NTBK at: https://dbc-5922e233-b716.cloud.databricks.com/ml/experiments/1690980704956355/runs/060bd78d2abc46c89fbf568a24551bb1
🧪 View experiment at: https://dbc-5922e233-b716.cloud.databricks.com/ml/experiments/1690980704956355

=== Comparación de modelos por val_f1 (desc) ===
                                           model_key  \
0                                    mlp_transformer   
1      bert_bert-base-multilingual-uncased-sentiment   
2  bert_distilbert-base-uncased-finetuned-sst-2-e...   

                             run_id  val_accuracy    val_f1  test_accuracy  \
0  060bd78d2abc46c89fbf568a24551bb1      0.659483  0.692607       0.691092   
1  58dda8810b2b4149969404d9a8d8ce1e      0.461207  0.390244       0.445402   
2  0b5158f11719429fa2796043612980b1      0.436782  0.357377       0.423851   

    test_f1  
0  0.714475  
1  0.371336  
2  0.350081  

🏆 Mejor modelo según val_f1: mlp_transformer (val_f1=0.6926)

✅ Alias 'champion' asignado 

In [30]:
# 4. Comparación consolidada y justificación final de modelos (visual + texto)

import plotly.express as px

# ---------------------------
# Construir tabla consolidada
# ---------------------------
comp_df = pd.DataFrame(RESULTS).copy()

# Añadir familia e hiperparámetros clave (a partir de model_key)
def enrich_row(row):
    mk = row["model_key"]
    if mk.startswith("bert_nlptown"):
        family = "bert_sequence_classifier"
        hyper = "hf_model=nlptown/bert-base-multilingual-uncased-sentiment"
        cost = "alto (modelo grande, multilingüe)"
        latency = "media-alta"
    elif mk.startswith("bert_distilbert"):
        family = "bert_sequence_classifier"
        hyper = "hf_model=distilbert-base-uncased-finetuned-sst-2-english"
        cost = "medio (modelo compacto)"
        latency = "media"
    elif mk == "mlp_transformer":
        family = "mlp_classifier + sentence_transformer"
        hyper = "ST=all-MiniLM-L6-v2, hidden=(256,), max_iter=10"
        cost = "medio (embeddings + MLP)"
        latency = "baja-media (si embeddings precalculados)"
    else:
        family = "desconocido"
        hyper = ""
        cost = ""
        latency = ""
    return pd.Series({"family": family, "hyperparams": hyper,
                      "cost_prod": cost, "latency_prod": latency})

extra_cols = comp_df.apply(enrich_row, axis=1)
comp_df = pd.concat([comp_df, extra_cols], axis=1)

# Señales simples de overfitting: gap entre val y test
comp_df["gap_f1_val_test"] = comp_df["val_f1"] - comp_df["test_f1"]
comp_df["gap_acc_val_test"] = comp_df["val_accuracy"] - comp_df["test_accuracy"]

# Ordenar por val_f1 desc
comp_df = comp_df.sort_values("val_f1", ascending=False).reset_index(drop=True)

print("=== Tabla consolidada de modelos (ordenada por val_f1 desc) ===")
display(comp_df[[
    "model_key", "family", "hyperparams",
    "val_accuracy", "val_f1", "test_accuracy", "test_f1",
    "gap_f1_val_test", "gap_acc_val_test",
    "cost_prod", "latency_prod"
]])

# ---------------------------
# Visualizaciones
# ---------------------------

# 1) Barras de F1 (validación y test) por modelo
long_f1 = comp_df.melt(
    id_vars=["model_key"],
    value_vars=["val_f1", "test_f1"],
    var_name="split",
    value_name="f1_score"
)

fig_f1 = px.bar(
    long_f1,
    x="model_key",
    y="f1_score",
    color="split",
    barmode="group",
    title="F1-score por modelo (validación vs test)",
)
fig_f1.show()

# 2) Barras de accuracy (validación y test) por modelo
long_acc = comp_df.melt(
    id_vars=["model_key"],
    value_vars=["val_accuracy", "test_accuracy"],
    var_name="split",
    value_name="accuracy"
)

fig_acc = px.bar(
    long_acc,
    x="model_key",
    y="accuracy",
    color="split",
    barmode="group",
    title="Accuracy por modelo (validación vs test)",
)
fig_acc.show()

# 3) Scatter val_f1 vs test_f1 para ver estabilidad
fig_scatter = px.scatter(
    comp_df,
    x="val_f1",
    y="test_f1",
    text="model_key",
    title="Val F1 vs Test F1 (estabilidad de desempeño)",
)
fig_scatter.update_traces(textposition="top center")
fig_scatter.show()

# ---------------------------
# Justificación final (texto)
# ---------------------------
best_row = comp_df.iloc[0]
best_model = best_row["model_key"]
best_family = best_row["family"]

print("\n=== Justificación final de modelo recomendado ===\n")
print(f"- Modelo con mejor F1 en validación: {best_model} "
      f"(val_f1={best_row['val_f1']:.4f}, test_f1={best_row['test_f1']:.4f})")
print(f"- Familia de modelo: {best_family}")
print(f"- Hiperparámetros clave: {best_row['hyperparams']}")
print(f"- Coste en producción (cualitativo): {best_row['cost_prod']}")
print(f"- Latencia esperada (cualitativa): {best_row['latency_prod']}")

gap_f1 = best_row["gap_f1_val_test"]
print(f"- Gap F1 val-test: {gap_f1:.4f} "
      "(positivo grande podría indicar overfitting).")

# Comentario de overfitting básico
if abs(gap_f1) < 0.02:
    print("  · Señal: el gap val-test es pequeño → desempeño estable, poco overfitting.")
elif gap_f1 > 0.05:
    print("  · Señal: el F1 en validación es bastante mejor que en test → posible overfitting.")
else:
    print("  · Señal: hay un gap moderado, revisar si es aceptable según el caso de uso.")




=== Tabla consolidada de modelos (ordenada por val_f1 desc) ===


,model_key,family,hyperparams,val_accuracy,val_f1,test_accuracy,test_f1,gap_f1_val_test,gap_acc_val_test,cost_prod,latency_prod
0,mlp_transformer,mlp_classifier + sentence_transformer,"ST=all-MiniLM-L6-v2, hidden=(256,), max_iter=10",0.659483,0.692607,0.691092,0.714475,-0.021868,-0.031609,medio (embeddings + MLP),baja-media (si embeddings precalculados)
1,bert_bert-base-multilingual-uncased-sentiment,desconocido,,0.461207,0.390244,0.445402,0.371336,0.018908,0.015805,,
2,bert_distilbert-base-uncased-finetuned-sst-2-e...,bert_sequence_classifier,hf_model=distilbert-base-uncased-finetuned-sst...,0.436782,0.357377,0.423851,0.350081,0.007296,0.012931,medio (modelo compacto),media



=== Justificación final de modelo recomendado ===

- Modelo con mejor F1 en validación: mlp_transformer (val_f1=0.6926, test_f1=0.7145)
- Familia de modelo: mlp_classifier + sentence_transformer
- Hiperparámetros clave: ST=all-MiniLM-L6-v2, hidden=(256,), max_iter=10
- Coste en producción (cualitativo): medio (embeddings + MLP)
- Latencia esperada (cualitativa): baja-media (si embeddings precalculados)
- Gap F1 val-test: -0.0219 (positivo grande podría indicar overfitting).
  · Señal: hay un gap moderado, revisar si es aceptable según el caso de uso.
